In [29]:
from collections import OrderedDict
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
from plotly.subplots import make_subplots
from EDA import char_movie_matrix, movie_lengths

In [4]:
def generar_grafico_barras_df(cantidad, no_serie, fases):
	
    columnas_seleccionadas = char_movie_matrix.columns[char_movie_matrix.loc['Fase'] .isin(fases)]
    data_f = char_movie_matrix[columnas_seleccionadas]
    if no_serie:
        # Filtrar las columnas excluyendo las que tienen False en la fila 'Serie'
        columnas_a_excluir = data_f.columns[data_f.loc['Serie'] == False]
        # Crear un nuevo DataFrame sin las columnas seleccionadas
        data = data_f[columnas_a_excluir]
    else:
        data = data_f
    
    data['Total de tiempo en pantalla'] = data.loc[:,:].sum(axis=1, skipna=True)
    data = data.sort_values(by='Total de tiempo en pantalla', ascending=False)
    data = data[data['Total de tiempo en pantalla'] != 0]
    data = data.head(cantidad)
    nombres_filas = data.index
    valores_ultima_columna = data.iloc[:, -1]

    # Crear un DataFrame con los datos
    df = pd.DataFrame({'Personaje': nombres_filas, 'Tiempo total en pantalla': valores_ultima_columna})

    # Crear el gráfico de barras con Plotly Express
    fig = px.bar(
        df,
        x='Personaje',
        y='Tiempo total en pantalla',
        labels={'Personaje': 'Personaje', 'Tiempo total en pantalla': 'Tiempo total en pantalla'},
        color_discrete_sequence=['rgb(179, 39, 14)'],  # Especifica el color de las barras
        hover_data={'Personaje': True, 'Tiempo total en pantalla': True},  # Incluye datos adicionales para mostrar en el hover
    )

    # Personalizar el formato del hovertemplate
    fig.update_traces(
        hovertemplate='Personaje: %{x}<br>Tiempo total en pantalla: %{y}'
    )

    # Ajustar diseño del gráfico
    fig.update_layout(height=500, width=900)

    # Devolver el objeto de figura (Figure) en lugar de mostrarlo
    return fig


In [26]:

def apariciones_pj(no_serie,fases):
    columnas_seleccionadas = char_movie_matrix.columns[char_movie_matrix.loc['Fase'] .isin(fases)]
    data_f = char_movie_matrix[columnas_seleccionadas]
    if no_serie:
        # Filtrar las columnas excluyendo las que tienen False en la fila 'Serie'
        columnas_a_excluir = data_f.columns[data_f.loc['Serie'] == False]
        # Crear un nuevo DataFrame sin las columnas seleccionadas
        data = data_f[columnas_a_excluir]
    else:
        data = data_f

    apariciones_pj = OrderedDict()

    for pj in data.index[0:-1]:
        # Filtrar las columnas que no son cero ni son de tipo string
        columnas_no_cero = data.loc[pj, (data.loc[pj] != 0)].index.tolist()
        # Excluir la última columna ('Serie') si existe
        apariciones_pj[pj] = columnas_no_cero
    return apariciones_pj



In [10]:


def generar_grafico_barras_num_apariciones(cantidad, fases, no_serie=False):
	
    ordenado_por_longitud = OrderedDict(sorted(apariciones_pj(no_serie,fases).items(), key=lambda x: len(x[1]), reverse=True))
    claves = list(ordenado_por_longitud.keys())[:cantidad]
    longitudes = [len(ordenado_por_longitud[clave]) for clave in claves]
	
    df = pd.DataFrame({'Claves': claves, 'Longitudes': longitudes})

    # Crear el gráfico de barras con Plotly Express
    fig = px.bar(
        df,
        x='Claves',
        y='Longitudes',
		title='Nº total de apariciones de los personajes',
        labels={'Claves': 'Claves', 'Longitudes': 'Longitudes'},
        color_discrete_sequence=['rgb(179, 39, 14)'],  # Especifica el color de las barras
        hover_data={'Personaje': True, 'Longitudes': True},  # Incluye datos adicionales para mostrar en el hover
    )

    # Personalizar el formato del hovertemplate
    fig.update_traces(
        hovertemplate='Personaje: %{x}<br>Nº de apariciones: %{y}'
    )

    # Ajustar diseño del gráfico
    fig.update_layout(height=500, width=900)
    # Devolver el objeto de figura (Figure) en lugar de mostrarlo
    return fig



In [50]:

def generar_grafico_peliculas_del_char(char,fases, no_serie=False):
    columnas_seleccionadas = char_movie_matrix.columns[char_movie_matrix.loc['Fase'] .isin(fases)]
    data_f = char_movie_matrix[columnas_seleccionadas]
	
    peliculas = {}
    for peli in apariciones_pj(no_serie,fases)[char]:
        peliculas[peli] = data_f.loc[char][peli]

    # Crear un DataFrame con los datos
    df = pd.DataFrame(list(peliculas.items()), columns=['Pelicula', 'Tiempo en pantalla'])
    # Calcular la suma de la columna 'Numeros'
    suma_total = df['Tiempo en pantalla'].sum()

    # Agregar una nueva fila con la suma total
    df = df.append({'Pelicula': 'Total', 'Tiempo en pantalla': suma_total}, ignore_index=True)

    # Crear el gráfico de barras con Plotly Express
    fig = px.bar(df,
                 x='Pelicula',
                 y='Tiempo en pantalla',
                 title=char,
                 color_discrete_sequence=['rgb(179, 39, 14)'],
                 labels={'Pelicula': 'Peliculas', 'Tiempo en pantalla': 'Tiempo en pantalla'},
				 hover_data={'Pelicula': True, 'Tiempo en pantalla': True},  # Incluye datos adicionales para mostrar en el hover
    )
    fig.update_traces(
        hovertemplate='Pelicula: %{x}<br>Tiempo en pantalla: %{y} mins'
    )
    # Ajustar diseño del gráfico
    fig.update_layout(height=400, width=800)
    # Devolver el objeto de figura (Figure) en lugar de mostrarlo
    return fig



In [79]:
def grafico_pie_t_pantalla (pelicula):    
    df = char_movie_matrix[char_movie_matrix[pelicula] != 0]
    df[pelicula] = pd.to_numeric(df[pelicula], errors='coerce')
    pie1 = df[pelicula]
    pie1_list = list(pie1.values)
    pie1_list = [round(each,2) for each in pie1_list]
    labels = df.index
	
    # Identificar las 5 mayores porciones
    top5_indices = df[pelicula].nlargest(5).index

# Crear la lista de etiquetas personalizadas
    custom_labels = [name if name in top5_indices else '' for name in labels]

    # Figure
    fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'pie'}]])

    # Convertir la paleta de colores a formato RGBA
    colores_rgba = list(reversed(px.colors.sequential.Reds))
    
    # Calcular los porcentajes manualmente
    porcentajes=[]
    for value in pie1_list:
        porcentajes.append(float(value)/float(movie_lengths['Eternals'])*100)#porcentajes = [(value / movie_lengths[pelicula]) * 100 for value in pie1_list]


    fig.add_trace(
        go.Pie(
            labels=labels,
            values=pie1_list,
            textinfo='text+value',
            hoverinfo="label+value",
            marker=dict(colors=colores_rgba),
            hole=0.3,
			text=custom_labels,
        ),
        row=1, col=1
        
    )

    fig.update_layout(
        title=dict(text="Tiempo en pantalla en " + pelicula + ' (' + movie_lengths[pelicula] +')',font=dict(size=24)),
        annotations=[
            {
                "font": {"size": 20},
                "showarrow": False,
                "text": pelicula,
                "x": 0.5,
                "y": 1.07,
            }
        ],
        height=700,  # Ajusta la altura del gráfico
        width=900,   # Ajusta el ancho del gráfico
        
    )
     # Actualizar las etiquetas con los porcentajes
    for i, label in enumerate(custom_labels):
        if label:
            custom_labels[i] += f' {porcentajes[i]:.2f}%'

    return fig

In [80]:
grafico_pie_t_pantalla('Eternals')


C:\Users\lache\AppData\Local\Temp\ipykernel_16560\2925245400.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
columnas_seleccionadas = char_movie_matrix.columns[char_movie_matrix.loc['Fase'] .isin([0,4])]
data_f = char_movie_matrix[columnas_seleccionadas]


,Bruja Escarlata y Visión,Falcon y el Soldado de Invierno,Loki,Viuda Negra,Shang-Chi y la leyenda de los diez anillos,Eternals,Ojo de Halcón,Spider-Man: No Way Home,Caballero Luna,Doctor Strange en el multiverso de la locura,Ms. Marvel,Thor: Love and Thunder,She-Hulk: Abogada Hulka,La maldición del hombre lobo,Black Panther: Wakanda Forever,Guardianes de la Galaxia: Especial felices fiestas
Tony Stark / Iron Man,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Pepper Potts,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Obadiah Stane,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Professor Ho Yinsen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
James Rhodes / War Machine,0.0,1.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Kevin Bacon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.5
Bzermiktoklok,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.75
Cosmo the Space Dog,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.25
Serie,True,True,True,False,False,False,True,False,True,False,True,False,True,False,False,False


In [19]:

apariciones_pj()

12.0